In [ ]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import sys
sys.path.append('ext')
import plot_util
import accelerometers

In [ ]:
inputdir = pathlib.Path.cwd() / 'inputdata'

In [ ]:
outputdir = pathlib.Path.cwd() / 'outputdata'
outputdir.mkdir(exist_ok=True)

In [ ]:
pathlib.Path.cwd()

In [ ]:
files = list(inputdir.glob('*/*/Accelerometer.csv'))
files

In [ ]:
for file in files:
    if len(files) > 5:
        suppress_figures = True
    else:
        suppress_figures = False
    df = accelerometers.clean_file(file)

    title = str(file.parents[1].stem) + '-- ' + str(file.parent.stem)
    print('\n', title)
    
    # Norm over time, in units of g-force
    plt.figure()
    plot_util.generic_plot(accelerometers.smooth(df[['g-force']], 100), kind='line', xlabel='Sec Elapsed', ylabel='g',
                           title=title + '_gforce', output_directory=outputdir)
    ps = accelerometers.ps(df, smooth_window=100)
    if suppress_figures:
        plt.close()

    # Log power spectrum
    plt.figure()
    plot_util.generic_plot(np.log10(ps[accelerometers.POWER_LABEL]), kind='line', xlabel='Hz', ylabel='log10(W/kg/Hz)',
                           title=title, output_directory=outputdir)
    if suppress_figures:
        plt.close()


    # Log power spectrum
    plt.figure()
    plot_util.generic_plot(np.log10(ps[accelerometers.POWER_LABEL]).rename(np.log10, axis=0), kind='line', xlabel='Hz', ylabel='log10(W/kg/Hz)',
                           title=title + '_logx', output_directory=outputdir)
    if suppress_figures:
        plt.close()
    
    total_pow = np.trapz(ps[accelerometers.POWER_LABEL], x=ps.index)
    print('Total Power: ' + str(int(total_pow)) + ' W/kg')
    print('Mean g-force: ' + str(df['g-force'].mean().round(2)) + " g's")
